# 获取 全国招投标 项目信息

以下代码主要由 GPT-4o 生成，演示 `BidsClient` 模块的版本演进。


## 使用 APISpace 提供的全国招投标查询信息API 获取项目信息

APISpace 为开发者提供了便捷获取 全国招投标项目信息 的相关 API。

链接如下：https://www.apispace.com/23330/api/project-info/apiDocument



## 身份认证

- APISpace 的所有 API 均使用 API 密钥来验证请求。您可以在 [访问控制页面](https://www.apispace.com/console/visitControll/?orgId=54392) 中查看和管理您的 API 密钥。

- 您的 API 密钥非常重要，因此请务必确保它们的安全！不要在可公开访问的区域（例如 GitHub、前端代码、客户端代码等）中存储或公开您的 API 密钥。

- 对 API 发出的所有请求都必须包含一个名为 “X-APISpace-Token” 的自定义 HTTP 头部（Header）。如果您的请求没有包含该自定义头部，或者该头部的Token参数不符合要求，则该 API 请求会失败。

- python 示例代码如下：
  ```python
     headers = {
                 "X-APISpace-Token": "ipvjsg2y557fii3*h5u627jfwq71imi4h3jX"  # 替换为您的 API 密钥
               }



## [v0.1] BidsClient Class 

In [10]:
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

class BidsClient:
    def __init__(self, api_token):
        self.api_token = api_token
        self.base_url = "https://23330.o.apispace.com/project-info/"
        self.headers = {
            "X-APISpace-Token": self.api_token,
            "Content-Type": "application/x-www-form-urlencoded"
        }

    def query_project_list(self, start_date, end_date, keyword, class_id, search_mode, search_type, page_index, page_size, province_code, city_code):
        """
        查询项目列表并返回Markdown格式，打印转换前的内容并提供调试提示
        """
        print(f"正在查询项目列表... 关键词: {keyword}, 查询时间范围: {start_date} 至 {end_date}")
        
        url = self.base_url + "project-list"
        payload = {
            "startDate": start_date,
            "endDate": end_date,
            "keyword": keyword,
            "classId": class_id,
            "searchMode": search_mode,
            "searchType": search_type,
            "pageIndex": page_index,
            "pageSize": page_size,
            "proviceCode": province_code,
            "cityCode": city_code
        }

        response = requests.post(url, data=payload, headers=self.headers)
        
        if response.status_code == 200:
            response_data = response.json()
            
            # 检查 API 返回的 code 是否为 200 (成功)
            if response_data.get("code") == 200:
                projects = response_data.get("data", {}).get("data", [])
                
                if projects:  # 检查项目列表是否为空
                    # 打印转换前的项目列表数据
                    print("\n--- 转换前的项目列表数据（原始数据） ---")
                    print(projects)
                    
                    print("\n--- 正在将项目列表数据转换为Markdown格式 ---\n")
                    
                    # 使用递归方法将项目列表数据转换为Markdown
                    md_content = f"# 项目列表 ({start_date} - {end_date})\n\n"
                    for project in projects:
                        md_content += dict_to_markdown(project)
                        md_content += "\n---\n"  # 用分隔符将不同项目分隔
                    
                    print("转换完成，返回Markdown内容。\n")
                    return md_content
                else:
                    print("查询成功，但返回的项目列表为空。")
            else:
                print(f"查询失败，API返回code: {response_data.get('code')}, 错误信息: {response_data.get('msg')}")
        else:
            print(f"请求失败，状态码: {response.status_code}, 错误信息: {response.text}")
        
        return None

    def query_project_details(self, publish_time, project_id):
        """
        查询项目详情并返回Markdown格式，打印转换前的内容并提供调试提示
        """
        print(f"正在查询项目详情... 项目ID: {project_id}, 发布时间: {publish_time}")
        
        url = self.base_url + "get-project"
        payload = {
            "publishTime": publish_time,
            "id": project_id
        }

        response = requests.post(url, data=payload, headers=self.headers)
        
        if response.status_code == 200:
            response_data = response.json()
            
            # 检查 API 返回的 code 是否为 200 (成功)
            if response_data.get("code") == 200:
                project_data = response_data.get("data", {})
                
                if project_data:  # 检查 data 是否为空
                    # 打印转换前的项目详情数据
                    print("\n--- 转换前的项目详情数据（原始数据） ---")
                    print(project_data)
                    
                    print("\n--- 正在将项目详情数据转换为Markdown格式 ---\n")
                    
                    # 使用递归方法将项目详情数据转换为Markdown
                    md_content = f"# 项目详情\n\n"
                    md_content += dict_to_markdown(project_data)
                    
                    print("转换完成，返回Markdown内容。\n")
                    return md_content
                else:
                    print("查询成功，但返回的 data 字段为空。")
            else:
                print(f"查询失败，API返回code: {response_data.get('code')}, 错误信息: {response_data.get('msg')}")
        else:
            print(f"请求失败，状态码: {response.status_code}, 错误信息: {response.text}")
        
        return None

        
    def query_project_structured_data(self, publish_time, project_id):
        """
        查询项目结构化数据并返回Markdown格式，打印转换前的内容并提供调试提示
        """
        print(f"正在查询项目结构化数据... 项目ID: {project_id}, 发布时间: {publish_time}")
        
        url = self.base_url + "getDetail"
        payload = {
            "publishTime": publish_time,
            "id": project_id
        }

        response = requests.post(url, data=payload, headers=self.headers)
        
        if response.status_code == 200:
            response_data = response.json()
            
            # 检查 API 返回的 code 是否为 200 (成功)
            if response_data.get("code") == 200:
                project_data = response_data.get("data", {})
                
                if project_data:  # 检查 data 是否为空
                    # 打印转换前的结构化数据
                    print("\n--- 转换前的结构化数据（原始数据） ---")
                    print(project_data)
                    
                    print("\n--- 正在将结构化数据转换为Markdown格式 ---\n")
                    
                    # 使用递归方法将项目结构化数据转换为Markdown
                    md_content = f"# 项目结构化数据\n\n"
                    md_content += dict_to_markdown(project_data)
                    
                    print("转换完成，返回Markdown内容。\n")
                    return md_content
                else:
                    print("查询成功，但返回的 data 字段为空。")
            else:
                print(f"查询失败，API返回code: {response_data.get('code')}, 错误信息: {response_data.get('msg')}")
        else:
            print(f"请求失败，状态码: {response.status_code}, 错误信息: {response.text}")
        
        return None


# Helper function to save Markdown content to file
def save_md_file(content, filename):
    # 确保目录 bid_info 存在
    dir_path = "bid_info"
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    file_path = os.path.join(dir_path, filename)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"文件已保存: {file_path}")

def dict_to_markdown(data, indent=0):
        """
        将字典数据递归转换为Markdown格式
        """
        md_content = ""
        for key, value in data.items():
            # 添加缩进和键
            md_content += " " * indent + f"**{key}**: "
            # 如果值是字典，递归处理
            if isinstance(value, dict):
                md_content += "\n" + dict_to_markdown(value, indent + 2)
            # 如果值是列表，列出所有项
            elif isinstance(value, list):
                md_content += "\n"
                for item in value:
                    md_content += " " * (indent + 2) + f"- {item}\n"
            else:
                # 普通值直接写入
                md_content += f"{value}\n"
        return md_content



### 实例化 BidsClient

从环境变量中获取 `BID_TOKEN`,实例化 `BidsClient`类

In [11]:
import os

# 设置环境变量 BID_TOKEN（确保该环境变量已在您的环境中设置）
api_token = os.getenv("BID_TOKEN")

if not api_token:
    raise ValueError("请设置 BID_TOKEN 环境变量")

# 实例化 BidsClient
client = BidsClient(api_token=api_token)


### 查询项目列表 API

API完整文档说明见：https://www.apispace.com/23330/api/project-info/apiDocument

**请求样例**：

```python

import requests

url = "https://23330.o.apispace.com/project-info/project-list"

payload = """
startDate=2022-01-01&
endDate=2022-12-30&
keyword=%E5%B7%A5%E7%A8%8B&
classId=2&
searchMode=1&
searchType=3&
pageIndex=1&
pageSize=5&
proviceCode=0&
cityCode=
"""

headers = {
    "X-APISpace-Token": "ipvjsg2y557fiihu627jfwq71imi4h3j",
    "Content-Type": ""
}

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)


```

**返回样例**：

```json
{
    "msg": "ok",
    "code": 200,
    "returnValue": null,
    "data": {
        "costtime": null,
        "data": [
            {
                "score": 0.9574359,
                "hasFile": 0,
                "newsTypeID": 2,
                "cityCode": "510100",
                "publish": "2023-01-18 22:41:27",
                "collectWebID": 1396,
                "proviceCode": "510000",
                "isHasFile": false,
                "id": 125829541,
                "title": "青海省某单位数字乡村试点支撑服务采购项目_中选结果公示"
            }
        ],
        "keyWordEntityType": null,
        "hasNext": true,
        "mess": "ok",
        "pageID": 1,
        "pageid": 1,
        "startdate": "2023-01-01 00:00:00",
        "maxCount": 3345,
        "total": 3345,
        "enddate": "2023-01-18 23:59:59",
        "state": 1,
        "seKeyWords": "采购"
    }
}

```

# 查询项目列表

In [12]:
# 查询项目列表并保存为Markdown文件
start_date = "2022-01-01"
end_date = "2022-12-30"
keyword = "工程"
class_id = "1"  # 1 表示招标信息
search_mode = "1"
search_type = "1"
page_index = "1"
page_size = "5"
province_code = "0"
city_code = ""

# 查询项目列表
project_list_md = client.query_project_list(start_date, end_date, keyword, class_id, search_mode, search_type, page_index, page_size, province_code, city_code)

# 如果查询成功，将其保存为Markdown文件
if project_list_md:
    current_date = datetime.now().strftime("%Y%m%d")
    filename = f"project_list_{start_date}_to_{end_date}_{current_date}.md"
    save_md_file(project_list_md, filename)
else:
    print("未能获取项目列表。")




正在查询项目列表... 关键词: 工程, 查询时间范围: 2022-01-01 至 2022-12-30

--- 转换前的项目列表数据（原始数据） ---
[{'cityCode': '431000', 'collectWebID': 62694, 'proviceCode': '430000', 'title': "<span style='color:red;'>工程</span>造价咨询服务-湖南天正项目管理有限公司直购定点公告", 'content': "采购单位: 苏仙区财政局 二、协议名称: <span style='color:red;'>工程</span>造价咨询服务,折扣率(%) 年度预算(元) 序号 服务,<span style='color:red;'>工程</span>造价咨询服务", 'score': 4.2753954, 'countyCode': None, 'hasFile': 0, 'newsTypeID': 1, 'projectClassID': None, 'publish': '2022-12-30 23:59:56', 'isHasFile': False, 'id': 124074996}, {'cityCode': '411600', 'collectWebID': 68733, 'proviceCode': '410000', 'title': '中铁三局天津公司沈丘项目采购绝缘子', 'content': "增值税专用发票 发票抬头:中铁三局集团天津建设<span style='color:red;'>工程</span>有限公司", 'score': 2.72757, 'countyCode': None, 'hasFile': 0, 'newsTypeID': 1, 'projectClassID': None, 'publish': '2022-12-30 23:59:12', 'isHasFile': False, 'id': 125174460}, {'cityCode': '621200', 'collectWebID': 16202, 'proviceCode': '620000', 'title': "西和县兴隆镇黑鹰村等10乡镇17处生态及地质灾害应急治理<span style='color:re


### 查询项目详情

In [13]:
# 查询项目详情并保存为Markdown文件
publish_time = "2023-01-18 22:41:27"  # 示例项目的发布时间
project_id = "125829541"  # 示例项目ID

# 查询项目详情
project_details_md = client.query_project_details(publish_time, project_id)

# 如果查询成功，将其保存为Markdown文件
if project_details_md:
    current_date = datetime.now().strftime("%Y%m%d")
    filename = f"project_details_{project_id}_{current_date}.md"
    save_md_file(project_details_md, filename)
else:
    print("未能获取项目详情。")



正在查询项目详情... 项目ID: 125829541, 发布时间: 2023-01-18 22:41:27

--- 转换前的项目详情数据（原始数据） ---
{'isFollowUp': 0, 'classid': 2, 'newsTypeID': 2, 'cityCode': '510100', 'publish': '2023-01-18 22:41:27', 'collectWebID': 1396, 'proviceCode': '510000', 'id': 125829541, 'title': '青海省某单位数字乡村试点支撑服务采购项目_中选结果公示', 'userID': 0, 'content': "<td colspan='2' id='contentInfo' ><div> <span>青海省某单位数字乡村试点支撑服务采购项目:标/包1的中选人为美讯慧云科技（成都）有限公司。</span><br/> <span>采购人/招标代理机构：中移系统集成有限公司/中国邮电器材集团有限公司</span><br/> <span>2023年01月18日</span><br/></div></td>", 'key': None}

--- 正在将项目详情数据转换为Markdown格式 ---

转换完成，返回Markdown内容。

文件已保存: bid_info\project_details_125829541_20240906.md


# 查询项目结构化数据

In [14]:
# 查询项目结构化数据并保存为Markdown文件
publish_time = "2023-01-18 22:41:27"  # 示例项目的发布时间
project_id = "125829541"  # 示例项目ID

# 查询项目结构化数据
project_structured_data_md = client.query_project_structured_data(publish_time, project_id)

# 如果查询成功，将其保存为Markdown文件
if project_structured_data_md:
    current_date = datetime.now().strftime("%Y%m%d")
    filename = f"project_structured_data_{project_id}_{current_date}.md"
    save_md_file(project_structured_data_md, filename)
else:
    print("未能获取项目结构化数据。")


正在查询项目结构化数据... 项目ID: 125829541, 发布时间: 2023-01-18 22:41:27

--- 转换前的结构化数据（原始数据） ---
{'partyAPhoneArr': [], 'bidMoneyString': None, 'partyBContactPersons': [], 'cityCode': '510100', 'proviceCode': '510000', 'agencyPhoneArr': [], 'collectUrl': 'https://b2b.10086.cn/b2b/main/viewNoticeContent.html?noticeBean.id=917359', 'budgetMoney': 0, 'partyAPerson': None, 'newsTypeID': 2, 'agencyContactPersons': [], 'partyAContactPersons': [], 'agencyPerson': None, 'partyBPhoneArr': ['19828257140'], 'projectID': 125829541, 'partyAName': None, 'publishTime': '2023-01-18 22:41:27', 'projectNumber': None, 'partyAPhone': None, 'budgetMoneyString': None, 'agencyName': '中移系统集成有限公司/中国邮电器材集团有限公司', 'partyBName': '美讯慧云科技（成都）有限公司', 'bidMoney': 0, 'agencyPhone': None, 'projectName': None}

--- 正在将结构化数据转换为Markdown格式 ---

转换完成，返回Markdown内容。

文件已保存: bid_info\project_structured_data_125829541_20240906.md
